# Wrangling Former Colonies
## By: Scott Kustes

### Objective:
Compile a list of modern countries that are former colonies of other modern countries, their colonizers, year of colonization, and year of independence.

#### Datasets:
The main dataset was downloaded from <a href='http://www.paulhensel.org/icow.html' target='_new'>The Issue Correlates of War Project</a>. Additional datasets were downloaded from <a href='http://www.correlatesofwar.org' target='_new'>The Correlates of War Project</a>.  For specific dataset citations, see the References section at the bottom of the notebook.

#### Contents
<ul>
    <li><a href='#gather'>Data Gathering</a></li>
    <li><a href='#assess'>Assess</a></li>
    <li><a href='#clean'>Clean</a></li>
    <li><a href='#final'>Finished Dataframes</a></li>
    <li><a href='#references'>References</a></li>
</ul>

In [5]:
# Import packages
import requests
import pandas as pd
import os.path as os_path

<a id='gather'></a>
## Data Gathering

In [6]:
icow = pd.read_csv( 'colonial_data.csv' )
icow.sample(5)

,State,Name,ColRuler,IndFrom,IndDate,IndViol,IndType,SecFrom,SecDate,SecViol,Into,IntoDate,COWsys,GWsys,Notes
126,510,Tanzania (Tanganyika),200,200,196112,0,2,-9,-9,-9,-9,-9,196112,196112,Tanganyika during mandate; united w/ Zanzibar ...
142,571,Botswana,200,200,196609,0,2,-9,-9,-9,-9,-9,196609,196609,-9
46,235,Portugal,-9,-9,113907,0,1,-9,-9,-9,-9,-9,181601,181601,Dependency of Spain 1580-1640
194,812,Laos,220,220,195310,1,2,-9,-9,-9,-9,-9,195310,195405,Claimed / invaded by Thais before French contr...
35,200,United Kingdom,-9,-9,106612,1,1,-9,-9,-9,-9,-9,181601,181601,-9


### Country Codes
Read in and de-duplicate the country codes used by The Correlates of War Project.

In [7]:
country_codes = pd.read_csv( 'cow_country_codes.csv' )
country_codes.drop_duplicates( inplace=True )
country_codes.sample(5)

,StateAbb,CCode,StateNme
75,CZR,316,Czech Republic
187,SAU,670,Saudi Arabia
17,SKN,60,St. Kitts and Nevis
41,NTH,210,Netherlands
216,MAD,781,Maldives


<a id='assess'></a>
## Assess

In [23]:
icow.query( 'ColRuler < 0' )['ColRuler'].unique()

,State,Name,ColRuler,IndFrom,IndDate,IndViol,IndType,SecFrom,SecDate,SecViol,Into,IntoDate,COWsys,GWsys,Notes


In [18]:
pd.set_option('display.max_colwidth', -1)
icow.query( 'ColRuler != IndFrom' )[['Name','IndDate','Notes']]

,Name,IndDate,Notes
3,Cuba,190205,COW system interrupted 9/1906-1/1909 (US occup); indep from USA coded as nonviolent (but long revolt against Spanish rule before USA acquired Cuba)
5,Dominican Republic,184402,COW system interrupted 11/1916-9/1924 (US occup); terr chg data incl 1865 indep from Spain; ruled by Haiti (not a COW system member) from 1822 to 1844 revolution
17,Guatemala,183904,Seceded from United States/Provinces of Cent.Am. (not in COW system; G&W 7/1/1823-12/31/1839)
18,Honduras,183811,Seceded from United States/Provinces of Cent.Am. (not in COW system; G&W 7/1/1823-12/31/1839)
19,El Salvador,184004,Seceded from United States/Provinces of Cent.Am. (not in COW system; G&W 7/1/1823-12/31/1839)
20,Nicaragua,183804,Seceded from United States/Provinces of Cent.Am. (not in COW system; G&W 7/1/1823-12/31/1839)
21,Costa Rica,183811,Seceded from United States/Provinces of Cent.Am. (not in COW system; G&W 7/1/1823-12/31/1839)
22,Panama,190311,Was part of Colombia until US supported 1903 rebellion
24,Venezuela,183001,Seceded from Gran Colombia (not in COW system; G&W: 8/30/1821-9/22/1830)
27,Ecuador,183005,Seceded from Gran Colombia (not in COW system; in G&W system 8/30/1821-9/22/1830)


### Issues Found
1) Remove unnecessary columns

2) Rename remaining columns

3) Remove countries with -9 in ColRuler: never colonized

4) Resolve disparities where ColRuler != IndFrom

<a id='clean'></a>
## Clean

<a id='references'></a>
## References

<li>Paul R. Hensel (2018). "ICOW Colonial History Data Set, version 1.1." Available at <a href='http://www.paulhensel.org/icowcol.html' target='_new'>http://www.paulhensel.org/icowcol.html</a></li>
<li><a href='http://www.correlatesofwar.org/data-sets/downloadable-files/cow-country-codes' target='_new'>Correlates of War country codes</a></li>